![Clothing Classifier Model](Clothing%20Classifier%20Model.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.


In [16]:
# Run the cells below first

In [17]:
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
from torchvision import datasets
import torchvision.transforms as transforms

In [19]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [20]:
classes = train_data.classes
num_classes = len(train_data.classes)

In [21]:
import numpy as np
import torch
from torchvision import datasets, transforms

# Load datasets
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Convert PyTorch tensor to NumPy array
images = train_data.data.numpy()
labels = np.array(train_data.targets)

# Display head of the data
head_images = images[:5]
head_labels = labels[:5]

print("Head of the data:")
print("Images shape:", head_images.shape)
print("Labels:", head_labels)

# Display tail of the data
tail_images = images[-5:]
tail_labels = labels[-5:]

print("\nTail of the data:")
print("Images shape:", tail_images.shape)
print("Labels:", tail_labels)


Head of the data:
Images shape: (5, 28, 28)
Labels: [9 0 0 3 0]

Tail of the data:
Images shape: (5, 28, 28)
Labels: [5 1 3 0 5]


In [22]:
from torchvision import datasets, transforms

# Load FashionMNIST dataset
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Get the list of classes
classes = train_data.classes

# Print the list of classes
print("Classes in the FashionMNIST dataset:")
for idx, class_name in enumerate(classes):
    print(f"{idx}: {class_name}")


Classes in the FashionMNIST dataset:
0: T-shirt/top
1: Trouser
2: Pullover
3: Dress
4: Coat
5: Sandal
6: Shirt
7: Sneaker
8: Bag
9: Ankle boot


In [23]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

In [24]:
class MultiClassImageClassifier(nn.Module):
  
    # Define the init method
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)
        
    def forward(self, x):
        # Pass inputs through each layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [25]:
# Define the training set DataLoader
dataloader_train = DataLoader(
    train_data,
    batch_size=10,
    shuffle=True,
)


In [26]:
# Define training function
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0
        num_processed = 0
        for features, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
        
    train_loss = running_loss / len(dataloader_train)


In [27]:
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr=0.01)

train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)


epoch 0, loss: 0.040509520426933884


In [28]:
#Test set Loader
dataloader_test = DataLoader(
    test_data,
    batch_size=10,
    shuffle=False,
)

In [29]:
#Now calculating evauation metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

In [30]:
#model runing on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy: (per_class)', accuracy)
print('Precision (per class):', precision)
print('Recall (per_class)', recall)

Accuracy: (per_class) 0.8673999905586243
Precision (per class): [0.7861060500144958, 0.9809428453445435, 0.8331527709960938, 0.7729116082191467, 0.7983281016349792, 0.9706477522850037, 0.7407407164573669, 0.9783352613449097, 0.9461020231246948, 0.8714788556098938]
Recall (per_class) [0.8600000143051147, 0.9779999852180481, 0.7689999938011169, 0.953000009059906, 0.7639999985694885, 0.9589999914169312, 0.5600000023841858, 0.8579999804496765, 0.9829999804496765, 0.9900000095367432]
